In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch as th
from data_preprocessing.ImageDataset import CustomImageDataset

In [4]:
dataset_path = Path("../data/datasets/dataset_10_images_per_class.pth")

if not dataset_path.exists():
    print("path does not exist")

dataset = th.load(dataset_path)

In [5]:
th_dataset = CustomImageDataset(dataset)
dataloader = DataLoader(th_dataset, batch_size=32, shuffle=True)